# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [5]:
%%R
df <- read_csv('2023_subway_censusgeo.csv')

Rows: 18208 Columns: 40
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (26): Common Name, Equipment Description, Executive Comment, Outage Cod...
dbl  (10): Outage, Station MRN, Station ID, Complex ID, lat, long, ADA, GEOI...
dttm  (4): Out of Service Date, Estimated Return to Service Date, Actual Ret...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
df = pd.read_csv('2023_subway_censusgeo.csv')

In [7]:

pd.set_option('display.max_columns', None)
df.head()


,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,lat,long,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference,GEOID,STATE,COUNTY,TRACT,BLOCK
0,2023-01-01 00:14:00,EL376,791238,ELE: EL376 - 068 - Bay Pkwy - Outside Area,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-01-01 07:00:00,2023-01-01 05:30:00,NaN,Phone,EE-MANOUT,2023-01-01 00:14:00,CL,***This Elevator is out of service for Mainten...,68,68,68,B21,BMT,West End,Bay Pkwy,Bk,D,Elevated,40.601875,-73.993728,Manhattan,Coney Island,1,NaN,POINT (-73.993728 40.601875),360470296002002,36,47,29600,2002
1,2023-01-01 01:04:00,EL189,791257,ELE: EL189 - 212 - Kingsbridge Rd - Outside Area,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,2023-01-01 09:00:00,2023-01-01 01:18:00,NaN,Phone,EE-MANOUT,2023-01-01 01:05:00,CL,\n\n***This elevator is out of service for Acc...,212,212,212,D04,IND,Concourse,Kingsbridge Rd,Bx,B D,Subway,40.866978,-73.893509,Bedford Pk Blvd & 205 St,Manhattan,1,NaN,POINT (-73.893509 40.866978),360050403021003,36,5,40302,1003
2,2023-01-01 02:01:00,EL293,791274,ELE: EL293 - 119 - 1 Av - Brooklyn Bound Platform,NaN,EUF,Closed,UPS Battery Failure,REPAIR,Repair,2023-01-01 11:00:00,2023-01-01 04:50:00,ST230101.txt - 192,External Monitoring System,EE-LNOUT,2023-01-01 02:08:00,CL,NaN,119,119,119,L06,BMT,Canarsie,1 Av,M,L,Subway,40.730953,-73.981628,8 Av,Brooklyn,1,NaN,POINT (-73.981628 40.730953),360610034004000,36,61,3400,4000
3,2023-01-01 02:08:00,EL449X,791275,ELE: EL449X - 279 - Sutphin Blvd-Archer Av - Mezz,NaN,TPE,Closed,NaN,REPAIR,Repair,2023-01-05 14:00:00,2023-01-05 03:06:00,NaN,Phone,EE-MANOUT,2023-01-01 02:09:00,CL,***This elevator is out of service as reported...,279,279,279,G06,IND,Queens - Archer,Sutphin Blvd-Archer Av-JFK Airport,Q,E J Z,Subway,40.700486,-73.807969,Jamaica Center,Manhattan,1,NaN,POINT (-73.807969 40.700486),360810208001000,36,81,20800,1000
4,2023-01-01 05:43:00,EL428,791299,ELE: EL428 - 273 - Queens Plaza - Outside Area,NaN,VAN,Closed,NaN,REPAIR,Repair,2023-01-01 13:00:00,2023-01-01 09:10:00,NaN,Phone,EE-MANOUT,2023-01-01 05:44:00,CL,NaN,273,273,273,G21,IND,Queens Blvd,Queens Plaza,Q,E M R,Subway,40.748973,-73.937243,Forest Hills - Jamaica,Manhattan,1,NaN,POINT (-73.937243 40.748973),360810033011019,36,81,3301,1019


## 👉 Grab Census Data

1. loading the Census API key

In [8]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [9]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

Loading required package: tidycensus
To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [10]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generally you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 46 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06011PR_001 Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 2 B06011PR_002 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 3 B06011PR_003 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 4 B06011PR_004 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 5 B06011PR_005 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 6 B06011_001   Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 7 B06011_002   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 8 B06011_003   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 9 B06011_004   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
10 B06011_005   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
11 B07011

In [11]:
%%R 
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
nyc_census_data <- get_acs(geography = "tract", 
                      state='NY',
                      county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                      variables = c(
                        population="B01003_001",
                        med_earn="B18140_002" # Median Earnings with Disability 
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=T)

nyc_census_data

  |======================================================================| 100%
Simple feature collection with 4654 features and 5 fields (with 2 geometries empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.4961 xmax: -73.70036 ymax: 40.91771
Geodetic CRS:  NAD83
First 10 features:
         GEOID                                       NAME   variable estimate
1  36081014700  Census Tract 147, Queens County, New York population     2863
2  36081014700  Census Tract 147, Queens County, New York   med_earn    44306
3  36047058400   Census Tract 584, Kings County, New York population     3655
4  36047058400   Census Tract 584, Kings County, New York   med_earn       NA
5  36061006900 Census Tract 69, New York County, New York population     2450
6  36061006900 Census Tract 69, New York County, New York   med_earn       NA
7  36047073000   Census Tract 730, Kings County, New York population     2676
8  36047073000   Census Tract 730, Kings County, N

Getting data from the 2017-2021 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'


## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [12]:
%%R

# pivot from long to wide
nyc_census_data <- nyc_census_data %>% 
  pivot_wider(
    names_from = variable, 
    values_from = c(estimate, moe),
    names_glue = "{variable}_{.value}"
  )
options(width = 1000)
nyc_census_data

Simple feature collection with 2327 features and 6 fields (with 1 geometry empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.4961 xmax: -73.70036 ymax: 40.91771
Geodetic CRS:  NAD83
# A tibble: 2,327 × 7
   GEOID       NAME                                                                                                                       geometry population_estimate med_earn_estimate population_moe med_earn_moe
   <chr>       <chr>                                                                                                            <MULTIPOLYGON [°]>               <dbl>             <dbl>          <dbl>        <dbl>
 1 36081014700 Census Tract 147, Queens County, New York   (((-73.9137 40.76548, -73.9121 40.76473, -73.9113 40.76435, -73.9105 40.76398, -73.9...                2863             44306            513        40118
 2 36047058400 Census Tract 584, Kings County, New York    (((-73.96103 40.59616, -73.95978 40.5963, -73.95878 40.

In [13]:
%%R

# drop rows with NA in column med_earn_estimate

nyc_census_data <- nyc_census_data %>%
    drop_na(med_earn_estimate)

nyc_census_data


Simple feature collection with 988 features and 6 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.5023 xmax: -73.70036 ymax: 40.91489
Geodetic CRS:  NAD83
# A tibble: 988 × 7
   GEOID       NAME                                                                                                                       geometry population_estimate med_earn_estimate population_moe med_earn_moe
   <chr>       <chr>                                                                                                            <MULTIPOLYGON [°]>               <dbl>             <dbl>          <dbl>        <dbl>
 1 36081014700 Census Tract 147, Queens County, New York   (((-73.9137 40.76548, -73.9121 40.76473, -73.9113 40.76435, -73.9105 40.76398, -73.9...                2863             44306            513        40118
 2 36047100400 Census Tract 1004, Kings County, New York   (((-73.90735 40.63083, -73.90415 40.63295, -73.90233 40.63132, -73.90051 40.62969,

In [14]:
%%R

# change df$GEOID to double
df$GEOID <- substr(df$GEOID, 1, 11)

nyc_census_data$GEOID <- as.numeric(nyc_census_data$GEOID)
df$GEOID <- as.numeric(df$GEOID)

# keep the first 11 digits in df$GEOID

# merge nyc_census_data with df on GEOID
df_census <- df %>%
    semi_join(nyc_census_data, by = c("GEOID" = "GEOID"))

In [15]:
%%R
df

# A tibble: 18,208 × 40
   `Out of Service Date` `Common Name` Outage `Equipment Description`                                  `Executive Comment` `Outage Code` Status `External Source Note`              `Reason Shown to Public` `Reason Shown to Public Description` `Estimated Return to Service Date` `Actual Return to Service Date` Reference          Source                     `Service Code` `Date Created`      `Status Code` `Outage Comments`                                                                                                                                                                                                                                                                                                                                                     `Station MRN` `Station ID` `Complex ID` `GTFS Stop ID` Division Line  `Stop Name` Borough `Daytime Routes` Structure   lat  long North Direction Labe…¹ South Direction Labe…²   ADA `ADA Notes` Georeference   GEOID 

In [16]:
%%R
df_census

# A tibble: 6,396 × 40
   `Out of Service Date` `Common Name` Outage `Equipment Description`                               `Executive Comment` `Outage Code` Status    `External Source Note`              `Reason Shown to Public` `Reason Shown to Public Description` `Estimated Return to Service Date` `Actual Return to Service Date` Reference           Source                     `Service Code` `Date Created`      `Status Code` `Outage Comments`                                                                                                                                       `Station MRN` `Station ID` `Complex ID` `GTFS Stop ID` Division Line               `Stop Name`              Borough `Daytime Routes` Structure   lat  long `North Direction Label`  `South Direction Label`       ADA `ADA Notes` Georeference                       GEOID STATE COUNTY TRACT  BLOCK
   <dttm>                <chr>          <dbl> <chr>                                                 <chr>               <chr>  

In [17]:
%%R
write_csv(df_census, "2023_subway_censusvar.csv")